In [1]:
import xarray as xr
import pandas as pd
from datetime import datetime
import os, glob
import numpy as np
from functools import reduce
import operator
import math
from math import pi

In [2]:
ds=xr.open_dataset("./IBTrACS.WP.v04r00.nc")
#筛选出1979-2021之间的台风
index_0=[]
index_1=[]
for i in range(4241):
    datetime_64=ds.time[i][0].values
    datetime_64=np.datetime64(datetime_64,'s')
    datetime_64=pd.to_datetime(datetime_64)
    date=datetime_64.strftime('%Y%m%d%H')
    if date[0:4]=='1979':
        index_0.append(i)
    if date[0:4]=='2022':
        index_1.append(i)
print(index_0[0],index_1[0])

2747 4238


In [3]:
with open(r'.\IBTrACS.txt', 'w') as outfile:
    for i in range(index_0[0],index_1[0]):
        lat=ds.lat[i].values
        lon=ds.lon[i].values
        time=ds.time[i].values
        num=0
        for j in range(len(lat)):
            if np.isnan(lat[j]):
                num=num+1
        ws1=ds.usa_wind[i].values
        ws2=ds.cma_wind[i].values
        ws3=ds.tokyo_wind[i].values
        p1=ds.usa_pres[i].values
        p2=ds.cma_pres[i].values
        p3=ds.tokyo_pres[i].values
        ws=[]
        p=[]
        for k in range(len(ws1)):
            if math.isnan(ws1[k])==False:
                speed=ws1[k]
            elif math.isnan(ws2[k])==False:
                speed=ws2[k]
            elif math.isnan(ws3[k])==False:
                speed=ws3[k]
            else:
                speed=ws1[k]
            if math.isnan(p1[k])==False:
                slp=p1[k]
            elif math.isnan(p2[k])==False:
                slp=p2[k]
            elif math.isnan(p3[k])==False:
                slp=p3[k]
            else:
                slp=p1[k]
            ws.append(speed)
            p.append(slp)
        ws=np.array(ws)
        p=np.array(p)
        speed=ds.storm_speed[i].values
        direct=ds.storm_dir[i].values
        name=ds.name[i].values
        data=pd.DataFrame({'names':name,'time':time,'lat':lat,'lon':lon,'ws':ws,'p':p,'speed':speed,'direct':direct})
        essemble=data.iloc[:(len(lat)-num)]
        outfile.write('66666,\n')
        np.savetxt(outfile, essemble, fmt = '%s,%s,%f,%f,%f,%f,%f,%f',delimiter = ',')
        print(i)

2747
2748
2749
2750
2751
2752
2753
2754
2755
2756
2757
2758
2759
2760
2761
2762
2763
2764
2765
2766
2767
2768
2769
2770
2771
2772
2773
2774
2775
2776
2777
2778
2779
2780
2781
2782
2783
2784
2785
2786
2787
2788
2789
2790
2791
2792
2793
2794
2795
2796
2797
2798
2799
2800
2801
2802
2803
2804
2805
2806
2807
2808
2809
2810
2811
2812
2813
2814
2815
2816
2817
2818
2819
2820
2821
2822
2823
2824
2825
2826
2827
2828
2829
2830
2831
2832
2833
2834
2835
2836
2837
2838
2839
2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2864
2865
2866
2867
2868
2869
2870
2871
2872
2873
2874
2875
2876
2877
2878
2879
2880
2881
2882
2883
2884
2885
2886
2887
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
2910
2911
2912
2913
2914
2915
2916
2917
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934
2935
2936
2937
2938
2939
2940
2941
2942
2943
2944
2945
2946


In [5]:
data_track=pd.read_csv(r'./IBTrACS.txt',sep=',',header=None,names=['name','date','lat','lon','ws','p','speed','direct'])
index_i=[]
for i in range(len(data_track)):
    if data_track.name[i]=='66666':
        index_i.append(i) 

In [6]:
#去掉nan值        
drop_index=[]
for i in range(len(index_i)):
    m=index_i[i]+1
    if i==len(index_i)-1:
        n=len(data_track)
    else:
        n=index_i[i+1]
    num=0
    for j in range(m,n):
        if np.isnan(data_track.ws[j]) or np.isnan(data_track.p[j]):
            num=num+1
            drop_index.append(j)
    if num==n-m:
        drop_index.append(m-1)
        
for i in range(len(drop_index)):
    data_track=data_track.drop(drop_index[i])

data_track.to_csv(r'./IBTrACS_dropnan.txt',index=False,header=False)

In [7]:
data_track=pd.read_csv(r'./IBTrACS_dropnan.txt',sep=',',header=None,names=['name','date','lat','lon','ws','p','speed','direct'])
index_i=[]
for i in range(len(data_track)):
    if data_track.name[i]=='66666':
        index_i.append(i) 
        
def datetime64(date_time):
    datetime=np.datetime64(date_time,'s')
    datetime_64=pd.to_datetime(datetime)
    date=datetime_64.strftime('%Y%m%d%H')
    date=int(date)
    return date

In [8]:
#去掉hour不是3小时间隔的
i_remove=[]
for i in range(len(index_i)):
    m=index_i[i]+1
    if i ==len(index_i)-1:
        n=len(data_track)
    else:
        n=index_i[i+1]
    sel_data=[]
    for l in range(m,n):
        lat=data_track.lat[l]
        lon=data_track.lon[l]
        date_time=data_track.date[l]      
        date=datetime64(date_time)
        datetime=pd.to_datetime(date,format='%Y%m%d%H')
        date_str=str(date)
        year=datetime.year     
        if date_str[8:10]=='06'or date_str[8:10]=='12'or date_str[8:10]=='18'or date_str[8:10]=='00'or date_str[8:10]=='03'or date_str[8:10]=='09'or date_str[8:10]=='15'or date_str[8:10]=='21':
            j=i+1
        else:
            i_remove.append(l)
for i in range(len(i_remove)):
    data_track=data_track.drop(i_remove[i])
data_track.to_csv(r'./IBTrACS_droperror.txt',index=False,header=False)

In [9]:
data_track=pd.read_csv(r'./IBTrACS_droperror.txt',sep=',',header=None,names=['name','date','lat','lon','ws','p','speed','direct'])
index_i=[]
for i in range(len(data_track)):
    if data_track.name[i]=='66666':
        index_i.append(i) 

In [10]:
#去掉时间上不连续的
i_remove=[]
for i in range(len(index_i)):
    m=index_i[i]+1
    if i ==len(index_i)-1:
        n=len(data_track)
    else:
        n=index_i[i+1]
    sel_data=[]
    length_mid=int((n-m)/2)
    for l in range(m,n-1):
        date_time0=data_track.date[l]
        date_time1=data_track.date[l+1]        
        date0=datetime64(date_time0)
        date1=datetime64(date_time1)
        datetime0=pd.to_datetime(date0,format='%Y%m%d%H')
        datetime1=pd.to_datetime(date0,format='%Y%m%d%H')
        date0=datetime64(date_time0)
        date_int0=int(date0)
        date_int1=int(date1)
        if date_int1-date_int0==3 or date_int1-date_int0==79 or date_int1-date_int0==7079 or date_int1-date_int0==7179 or date_int1-date_int0==6979 or date_int1-date_int0==886979:
            j=i+1
        else:
            if l<=length_mid:
                i_remove=i_remove+[h for h in range(m,l+2)]
            else:
                i_remove=i_remove+[h for h in range(l,n)]                
i_remove_list=list(set(i_remove))
for i in range(len(i_remove_list)):
    data_track=data_track.drop(i_remove_list[i])
data_track.to_csv(r'./IBTrACS_droptime.txt',index=False,header=False)

In [11]:
data_track=pd.read_csv(r'./IBTrACS_droptime.txt',sep=',',header=None,names=['name','date','lat','lon','ws','p','speed','direct'])
index_i=[]
for i in range(len(data_track)):
    if data_track.name[i]=='66666':
        index_i.append(i) 

In [12]:
#去掉生命史小于96个小时
remove=[]
for i in range(len(index_i)):
    m=index_i[i]+1
    if i==len(index_i)-1:
        n=len(data_track)
    else:
        n=index_i[i+1]
    if n-m<33:
        for j in range(m-1,n):
            remove.append(j)
for i in range(len(remove)):
    data_track=data_track.drop(remove[i])
data_track.to_csv(r'./IBTrACS_fore72.txt',index=False,header=False)

In [18]:
# remove_rmtime=[]
# for i in range(len(index_i)):
#     j=index_i[i]+1
#     if i==len(index_i)-1:
#         k=len(data_track)
#     else:
#         k=index_i[i+1]
#     for m in range(j,k):
#         if data_track.lon[m]>170 or data_track.lat[m]>60 or data_track.lon[m]<10 or data_track.lat[m]<10 :
#             remove_rmtime.append(m) 

# for i in range(len(remove_rmtime)):
#     data_track=data_track.drop(remove_rmtime[i])

# data_track.to_csv(r'G:/typhoon-ocean/last_test/IBTrACS_dropnan_rm180.txt',index=False,header=False)